In [1]:
import pickle
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from rdkit import Chem

import shap
shap.explainers._deep.deep_tf.op_handlers['AddV2'] = shap.explainers._deep.deep_tf.passthrough

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#from tensorflow.compat.v1.keras.backend import get_session
#tf.compat.v1.disable_v2_behavior()
#tf.compat.v1.enable_eager_execution()

In [2]:
def shap_CN(num_atoms_of_interest):
    df = pd.read_csv('molecules_to_predict.csv')
    df['total_atoms'] = [ Chem.MolFromSmiles(smi).GetNumHeavyAtoms() for smi in df.Canonical_SMILES]

    with open('weights_last_layer.pkl','rb') as f:
        #CN_readout, norelu, relu
        w1, w2, w3 = pickle.load(f)

    with open('feat_vectors_for_shap.pkl','rb') as f:
        af_pkl, gf_pkl = pickle.load(f)

    indices_of_interest = df[ df['total_atoms'] == num_atoms_of_interest ].index
    af_pkl = af_pkl[indices_of_interest][:, 0:num_atoms_of_interest]
    gf_pkl = gf_pkl[indices_of_interest]        
    df = df[ df['total_atoms'] == num_atoms_of_interest ]
    
    #af_input = layers.Input(shape=[num_atoms_of_interest,64], dtype=tf.float32, name='af')
    af_input = layers.Input(shape=[64], dtype=tf.float32, name='af')
    gf_input = layers.Input(shape=[64], dtype=tf.float32, name='gf')

    #af = layers.GlobalAveragePooling1D()(af_input)
    af = layers.Dense(64, activation='relu', name = 'denserelu')(af_input)
    af = layers.Dense(64, name = 'dense')(af)
    gf = layers.Add()([gf_input, af])
    prediction = layers.Dense(1, name = 'dense_final')(gf)
    
    input_tensors = [af_input, gf_input]
    model = tf.keras.Model(input_tensors, [prediction])
    
    model.layers[-1].set_weights( [w1[0].numpy(), w1[1].numpy()] )
    model.layers[-3].set_weights( [w2[0].numpy(), w2[1].numpy()] )
    model.layers[-5].set_weights( [w3[0].numpy(), w3[1].numpy()] )

    af_pkl_avg = np.mean(af_pkl, axis = 1)
    pred = model.predict([af_pkl_avg, gf_pkl]).squeeze()
    
    
    
    #### SHAP part ####
    e = shap.DeepExplainer(model, [af_pkl_avg, gf_pkl])
    shap_values = e.shap_values([af_pkl_avg, gf_pkl], check_additivity=False)
    
    af_shap, gf_shap = shap_values[0]
    all_shap = af_shap + gf_shap
    
    atomwise_shap = np.zeros((len(df), num_atoms_of_interest, 64))
    for i in range(len(af_pkl)): # af: (Num_atoms_in_a_molecule * 64)
        for j in range(len(af_pkl[i])):  # Num_atoms_in_a_molecule
            for k in range(len(af_pkl[i][j])): # 64
                #atomwise_shap[i][j][k] = (af_pkl[i][j][k] / (num_atoms_of_interest * af_pkl_avg[i][k])) * af_shap[i][k]    
                atomwise_shap[i][j][k] = (af_pkl[i][j][k] / (num_atoms_of_interest * af_pkl_avg[i][k])) * all_shap[i][k]                
    atomwise_shap_not_normalized = np.sum(atomwise_shap, axis = 2)
    
    # to find the ax+b correlation between af_shap + gf_shap vs. predicted CN
    af_shap_summed = np.sum(af_shap, axis = 1)
    gf_shap_summed = np.sum(gf_shap, axis = 1)
    
    CN_shap = af_shap_summed + gf_shap_summed
    
    reg = LinearRegression().fit(CN_shap.reshape(-1,1), df.predicted)
    
    
    a, b  = reg.coef_[0], reg.intercept_
    atomwise_shap_normalized_to_CN = atomwise_shap_not_normalized + b / num_atoms_of_interest
    
    #atomwise_shap_normalized_to_CN = np.multiply(atomwise_shap_for_plot,
    #                                              np.tile(
    #                                                  np.expand_dims(
    #                                                      np.divide(  np.array(df.predicted), 
    #                                                                  np.sum(atomwise_shap_for_plot, axis = -1)
    #                                                               ), 
    #                                                   axis = 1),
    #                                              num_atoms_of_interest))
    #atom_shap_total = np.sum(atomwise_shap_normalized_to_CN, axis = -1)
    #return df, atomwise_shap_normalized_to_CN, atom_shap_total

    print(np.abs(pred -  df.predicted).mean(), np.abs(pred -  df.predicted).max())
    print(num_atoms_of_interest, reg.coef_, reg.intercept_, reg.score(CN_shap.reshape(-1,1), df.predicted))
    print('!!!!!!!!', np.abs(np.sum(atomwise_shap_normalized_to_CN, axis = -1) -  df.predicted).mean())
    
    return df, atomwise_shap_normalized_to_CN, np.sum(atomwise_shap_normalized_to_CN, axis = -1)

In [3]:
df = pd.read_csv('molecules_to_predict.csv')
df['total_atoms'] = [ Chem.MolFromSmiles(smi).GetNumHeavyAtoms() for smi in df.Canonical_SMILES]
df_w_shap = pd.DataFrame(columns = list(df.columns) + ['atom_shap_total','atomwise_shap'])

#for i in range(8,25):
for i in range(2,43): #all atoms
#for i in range(2,3): 
    if len(df[df['total_atoms']  == i]) == 0:
        continue
    df_sub, atomwise_shap, atom_shap_total = shap_CN(i)
    df_sub['atomwise_shap'] = list(atomwise_shap)
    df_sub['atom_shap_total'] = list(atom_shap_total)
    df_w_shap = pd.concat([df_w_shap, df_sub])

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


0.0008434871337890115 0.0008434871337890115
2 [0.] 5.7731423 nan
!!!!!!!! 0.0


R^2 score is not well-defined with less than two samples.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


0.009726522448729735 0.01923083496093625
3 [1.00028527] 42.0811243 1.0
!!!!!!!! 0.009726079179281033


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


...

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


0.006770610351559014 0.006770610351559014
40 [0.] 60.775105 nan
!!!!!!!! 7.105427357601002e-15


R^2 score is not well-defined with less than two samples.


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


0.014035511108399845 0.020628586425786466
42 [0.99973704] 67.469979 1.0
!!!!!!!! 0.006592857339121849


In [4]:
'''
import matplotlib.pyplot as plt
import matplotlib

import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})
%matplotlib inline

matplotlib.rcParams['figure.dpi'] = 300
#plt.rcParams["font.family"] = 'Arial'
#plt.rcParams.update({'font.size': 24})

plt.scatter(af_shap_summed + gf_shap_summed, df.predicted)
'''

'\nimport matplotlib.pyplot as plt\nimport matplotlib\n\nimport seaborn as sns\nsns.set(context=\'talk\', style=\'ticks\',\n        color_codes=True, rc={\'legend.frameon\': False})\n%matplotlib inline\n\nmatplotlib.rcParams[\'figure.dpi\'] = 300\n#plt.rcParams["font.family"] = \'Arial\'\n#plt.rcParams.update({\'font.size\': 24})\n\nplt.scatter(af_shap_summed + gf_shap_summed, df.predicted)\n'

In [5]:
df_w_shap

,Canonical_SMILES,Device_tier,Train/Valid/Test,CN,predicted,glob_vector,total_atoms,atom_shap_total,atomwise_shap
533,CO,1,Valid,6.10,5.773142,0.577515 2.314571 -0.284544 0.471034 1.244606 ...,2,5.773142,"[2.88657115, 2.88657115]"
213,CCO,1,Train,8.00,7.976499,0.568872 2.785593 -0.540729 -0.305534 0.827454...,3,7.986225,"[4.397712684170854, -0.6515128485892792, 4.240..."
371,COC,3,Train,67.00,76.185750,2.974701 19.654045 -12.175194 -6.312239 3.1758...,3,76.176024,"[22.183926446684424, 31.808170912773832, 22.18..."
344,CCCO,1,Train,8.20,8.250709,0.380596 2.478370 -0.743939 -0.695015 0.493383...,4,8.247726,"[2.245984029826436, 0.44175407559303004, 2.384..."
587,CCCC,1,Test,20.62,28.108671,1.248712 7.012758 -4.568620 -2.607537 1.467168...,4,28.111654,"[6.360028955270834, 7.695797930115594, 7.69579..."
...,...,...,...,...,...,...,...,...,...
558,CCCCCCCCOC(=O)CCCCCCCCC(=O)OCCCCCCCC,3,Valid,70.00,96.371620,5.207996 26.726585 -14.537582 -5.733907 4.9977...,30,96.366214,"[3.205967626297348, 3.135653605606199, 3.24326..."
44,CCCCCCCCCCCCCCCC(=O)OC.CCCCCCCCCCCCCCCC(=O)OC,1,Train,85.90,83.955190,4.558927 23.319794 -12.815701 -5.431929 4.2278...,38,83.955190,"[2.209347105263158, 2.209347105263158, 2.20934..."
297,CCCCCCCCCCCCCCCCCC(=O)O.CCCCCCCCCCCCCCCCCC(=O)O,1,Train,61.70,60.775105,3.533552 16.269886 -9.057496 -5.823801 3.54190...,40,60.775105,"[1.5193776250000002, 1.5193776250000002, 1.519..."
294,CCCCC/C=C\C/C=C\CCCCCCCC(=O)OC.CCCCC/C=C\C/C=C...,1,Train,43.90,42.405098,1.464636 10.907668 -7.314800 -2.568479 0.80509...,42,42.398506,"[1.2347154124631423, 1.1024885877352995, 0.712..."


In [6]:
df_w_shap.to_csv('CN_shap_250213.csv', index = False)